In [1]:
from datetime import date
from datetime import datetime
import time
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import re

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
    
# source: https://stackoverflow.com/questions/3368969/find-string-between-two-substrings

In [3]:
# f = open("console_output_2.txt", "a")
# now = datetime.now()
# current_time = now.strftime("%H:%M:%S")
# f.write("file open at =" + current_time + "\n")

In [4]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("program start at", current_time)

program start at 16:46:20


In [5]:
allUrl = "http://www.nuforc.org/webreports/ndxloc.html"
allPage = urllib.request.urlopen(allUrl)
allSoup = BeautifulSoup(allPage, 'html.parser')

states = {
    'AK': 'ALASKA',
    'AL': 'ALABAMA',
    'AR': 'ARKANSAS',
    'AZ': 'ARIZONA',
    'CA': 'CALIFORNIA',
    'CO': 'COLORADO',
    'CT': 'CONNECTICUT',
    'DC': 'DISTRICT OF COLUMBIA',
    'DE': 'DELAWARE',
    'FL': 'FLORIDA',
    'GA': 'GEORGIA',
    'HI': 'HAWAII',
    'IA': 'IOWA',
    'ID': 'IDAHO',
    'IL': 'ILLINOIS',
    'IN': 'INDIANA',
    'KS': 'KANSAS',
    'KY': 'KENTUCKY',
    'LA': 'LOUISIANA',
    'MA': 'MASSACHUSETTS',
    'MD': 'MARYLAND',
    'ME': 'MAINE',
    'MI': 'MICHIGAN',
    'MN': 'MINNESOTA',
    'MO': 'MISSOURI',
    'MS': 'MISSISSIPPI',
    'MT': 'MONTANA',
    'NC': 'NORTH CAROLINA',
    'ND': 'NORTH DAKOTA',
    'NE': 'NEBRASKA',
    'NH': 'NEW HAMPSHIRE',
    'NJ': 'NEW JERSEY',
    'NM': 'NEW MEXICO',
    'NV': 'NEVADA',
    'NY': 'NEW YORK',
    'OH': 'OHIO',
    'OK': 'OKLAHOMA',
    'OR': 'OREGON',
    'PA': 'PENNSYLVANIA',
    'RI': 'RHODE ISLAND',
    'SC': 'SOUTH CAROLINA',
    'SD': 'SOUTH DAKOTA',
    'TN': 'TENNESSEE',
    'TX': 'TEXAS',
    'UT': 'UTAH',
    'VA': 'VIRGINIA',
    'VT': 'VERMONT',
    'WA': 'WASHINGTON',
    'WI': 'WISCONSIN',
    'WV': 'WEST VIRGINIA',
    'WY': 'WYOMING'
}

# thanks to https://gist.github.com/JeffPaine/3083347

table = allSoup.find("table")

stateLinks = []
for link in table.find_all("a"):
    if link.get_text() in states.values():
        stateLinks.append(link['href'])

reportLinks = []
for state in stateLinks:
    stateUrl = 'http://www.nuforc.org/webreports/' + state
    statePage = urllib.request.urlopen(stateUrl)
    stateSoup = BeautifulSoup(statePage, 'html.parser')
    stateTable = stateSoup.find("tbody")
    for x in stateTable.find_all("tr"):
        link = x.find("a")['href']
        reportLinks.append('http://www.nuforc.org/webreports/' + link)

In [6]:
len(reportLinks)

122754

In [7]:
rows = []

now = datetime.now()
loop_start = now
current_time = now.strftime("%H:%M:%S")
print("going into loop at ", current_time)

counter = 0

for link in reportLinks:
    reportPage = urllib.request.urlopen(link)
    reportSoup = BeautifulSoup(reportPage, 'html.parser')
    report = reportSoup.find("tbody").find_all("td")
    text = list(report[0].stripped_strings)
    textString = " ".join(text)
    occured = ""
    reported = ""
    posted = ""
    town = ""
    state = ""
    shape = ""
    duration = ""
    characteristics = ""
    
    for line in text:
        if "Occurred" in line:
            occurred = find_between(line, ":", "(").strip()
        if "Reported" in line:
            reported_date = find_between(line, ": ", " ").strip()
            if "AM" in line:
                reported_time = line.split("AM")[1].strip()
            if "PM" in line:
                reported_time = line.split("PM")[1].strip()
            reported = reported_date + " " + reported_time
        if "Posted" in line:
            posted = line.split(":")[1].strip()
        if "Location" in line:
            location = line.split(":")[1].strip()
            local = location.split(", ")
            town = local[0].strip()
            if len(local) > 1:
                state = local[1].strip()
        if "Shape" in line:
            shape = line.split(":")[1].strip()
        if "Duration" in line:
            duration = line.split(":")[1].strip()
        if "Characteristic" in line:
            characteristics = line.split(":")[1].strip()
      
    summary = report[1].get_text().strip()
    
    img_urls = []
    img_links = reportSoup.find_all("img", {"class": "ufopic"})
    for i in img_links: 
        img_urls.append(i["src"])
    
    reportRow = [link, occurred, reported, posted, town, state, "USA", shape, 
                 duration, characteristics, summary, img_urls]
    rows.append(reportRow)
    
    if counter % 1000 == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("counter at ", counter, " at ", current_time)
    counter+= 1
    
    

going into loop at  16:51:03
counter at  0  at  16:51:03
counter at  1000  at  16:52:37
counter at  2000  at  16:54:10
counter at  3000  at  16:55:49
counter at  4000  at  16:57:25
counter at  5000  at  16:59:05
counter at  6000  at  17:00:37
counter at  7000  at  17:02:15
counter at  8000  at  17:03:51
counter at  9000  at  17:05:29
counter at  10000  at  17:07:07
counter at  11000  at  17:08:41
counter at  12000  at  17:10:17
counter at  13000  at  17:11:56
counter at  14000  at  17:13:35
counter at  15000  at  17:15:21
counter at  16000  at  17:16:59
counter at  17000  at  17:18:38
counter at  18000  at  17:20:08
counter at  19000  at  17:21:41
counter at  20000  at  17:23:13
counter at  21000  at  17:24:46
counter at  22000  at  17:26:21
counter at  23000  at  17:28:00
counter at  24000  at  17:29:37
counter at  25000  at  17:31:14
counter at  26000  at  17:32:49
counter at  27000  at  17:34:23
counter at  28000  at  17:36:00
counter at  29000  at  17:37:40
counter at  30000  at  1

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


counter at  115000  at  19:51:10
counter at  116000  at  19:52:46
counter at  117000  at  19:54:21
counter at  118000  at  19:55:50
counter at  119000  at  19:57:20
counter at  120000  at  19:58:51
counter at  121000  at  20:00:28
counter at  122000  at  20:02:12


In [9]:
now = datetime.now()
loop_end = now
current_time = now.strftime("%H:%M:%S")
print("loop done at", current_time)

print("time lapsed: ", loop_start, loop_end)

loop done at 20:08:40
time lapsed:  16:51:03 2022-05-24 20:08:40.438439


In [10]:
df = pd.DataFrame(rows, 
             columns=["Link", "Occurred", "Reported", "Posted", "City", "State", 
                      "Country", "Shape", "Duration", "Characteristics", "Summary", "Images"])

In [14]:
df.to_csv('../big-ufo-data/nuforcData_2022.csv')
df.to_json('../big-ufo-data/nuforcData_2022.json', orient="records")